<a href="https://colab.research.google.com/github/Sanvee0306/DS/blob/main/Recommendation_Engine_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#UBCF
import pandas as pd
import numpy as np

In [3]:
movies_df = pd.read_csv('Movie (1).csv')

In [4]:
movies_df[0:5]

,userId,movie,rating
0,3,Toy Story (1995),4.0
1,6,Toy Story (1995),5.0
2,8,Toy Story (1995),4.0
3,10,Toy Story (1995),4.0
4,11,Toy Story (1995),4.5


In [5]:
#number of unique users in the dataset
len(movies_df.userId.unique())

4081

In [6]:
len(movies_df.movie.unique())

10

In [8]:
user_movies_df = movies_df.pivot(index='userId',
                                 columns='movie',
                                 values='rating').reset_index(drop=True)

In [10]:
user_movies_df

movie,Father of the Bride Part II (1995),GoldenEye (1995),Grumpier Old Men (1995),Heat (1995),Jumanji (1995),Sabrina (1995),Sudden Death (1995),Tom and Huck (1995),Toy Story (1995),Waiting to Exhale (1995)
0,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
3,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4076,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4077,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
4078,NaN,3.0,4.0,5.0,NaN,3.0,1.0,NaN,4.0,NaN
4079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN


In [11]:
user_movies_df.index = movies_df.userId.unique()

In [12]:
user_movies_df

movie,Father of the Bride Part II (1995),GoldenEye (1995),Grumpier Old Men (1995),Heat (1995),Jumanji (1995),Sabrina (1995),Sudden Death (1995),Tom and Huck (1995),Toy Story (1995),Waiting to Exhale (1995)
3,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
10,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7044,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7070,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
7080,NaN,3.0,4.0,5.0,NaN,3.0,1.0,NaN,4.0,NaN
7087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN


In [13]:
#Impute those NaNs with 0 values
user_movies_df.fillna(0, inplace=True)

In [14]:
user_movies_df

movie,Father of the Bride Part II (1995),GoldenEye (1995),Grumpier Old Men (1995),Heat (1995),Jumanji (1995),Sabrina (1995),Sudden Death (1995),Tom and Huck (1995),Toy Story (1995),Waiting to Exhale (1995)
3,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
10,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7044,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7070,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
7080,0.0,3.0,4.0,5.0,0.0,3.0,1.0,0.0,4.0,0.0
7087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0


In [15]:
#Calculating Cosine Similarity between Users
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

In [16]:
user_sim = 1 - pairwise_distances( user_movies_df.values,metric='cosine')

In [17]:
user_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.55337157],
       [0.        , 1.        , 0.        , ..., 0.45883147, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.45883147, 1.        ,
        0.62254302],
       ...,
       [0.        , 0.45883147, 0.45883147, ..., 1.        , 0.45883147,
        0.47607054],
       [0.        , 0.        , 1.        , ..., 0.45883147, 1.        ,
        0.62254302],
       [0.55337157, 0.        , 0.62254302, ..., 0.47607054, 0.62254302,
        1.        ]])

In [18]:
#Store the results in a dataframe
user_sim_df = pd.DataFrame(user_sim)

In [20]:
#Set the index and column names to user ids 
user_sim_df.index = movies_df.userId.unique()
user_sim_df.columns = movies_df.userId.unique()

In [21]:
user_sim_df.iloc[0:5, 0:5]

,3,6,8,10,11
3,1.0,0.0,0.0,0.0,1.0
6,0.0,1.0,0.0,0.0,0.0
8,0.0,0.0,1.0,0.0,0.0
10,0.0,0.0,0.0,1.0,0.0
11,1.0,0.0,0.0,0.0,1.0


In [22]:
np.fill_diagonal(user_sim, 0)
user_sim_df.iloc[0:5, 0:5]

,3,6,8,10,11
3,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0
11,1.0,0.0,0.0,0.0,0.0


In [23]:
#Most Similar Users
user_sim_df.idxmax(axis=1)[0:5]

3       11
6      168
8       16
10    4047
11       3
dtype: int64

In [26]:
movies_df[(movies_df['userId']==6) | (movies_df['userId']==168)]

,userId,movie,rating
1,6,Toy Story (1995),5.0
60,168,Toy Story (1995),4.5
3725,6,Grumpier Old Men (1995),3.0
6464,6,Sabrina (1995),5.0


In [27]:
user_1=movies_df[movies_df['userId']==6] 

In [28]:
user_2=movies_df[movies_df['userId']==11]

In [29]:
user_2.movie

4       Toy Story (1995)
7446    GoldenEye (1995)
Name: movie, dtype: object

In [30]:
user_1.movie

1              Toy Story (1995)
3725    Grumpier Old Men (1995)
6464             Sabrina (1995)
Name: movie, dtype: object

In [31]:
pd.merge(user_1,user_2,on='movie',how='outer')

,userId_x,movie,rating_x,userId_y,rating_y
0,6.0,Toy Story (1995),5.0,11.0,4.5
1,6.0,Grumpier Old Men (1995),3.0,NaN,NaN
2,6.0,Sabrina (1995),5.0,NaN,NaN
3,NaN,GoldenEye (1995),NaN,11.0,2.5


In [32]:
#IBCF
#Fetching ratings for Toy Story
Toy_user_rating = user_movies_df['Sabrina (1995)']  

In [33]:
Toy_user_rating

3       0.0
6       0.0
8       0.0
10      0.0
11      0.0
       ... 
7044    0.0
7070    0.0
7080    3.0
7087    0.0
7105    4.0
Name: Sabrina (1995), Length: 4081, dtype: float64

In [34]:
#Finding the correlation with different movies
similar_to_Toy = user_movies_df.corrwith(Toy_user_rating) 


In [35]:
similar_to_Toy.head(10) 

movie
Father of the Bride Part II (1995)    0.287343
GoldenEye (1995)                     -0.021940
Grumpier Old Men (1995)               0.277298
Heat (1995)                           0.059740
Jumanji (1995)                        0.054658
Sabrina (1995)                        1.000000
Sudden Death (1995)                   0.130281
Tom and Huck (1995)                   0.123087
Toy Story (1995)                      0.002798
Waiting to Exhale (1995)              0.132838
dtype: float64

In [36]:

corr_Toy = pd.DataFrame(similar_to_Toy, columns=['Correlation'])
corr_Toy.dropna(inplace=True)
corr_Toy.head()

,Correlation
movie,
Father of the Bride Part II (1995),0.287343
GoldenEye (1995),-0.021940
Grumpier Old Men (1995),0.277298
Heat (1995),0.059740
Jumanji (1995),0.054658


In [37]:
corr_Toy[corr_Toy['Correlation'] > 0].sort_values(by='Correlation', ascending=False).head(5)  

,Correlation
movie,
Sabrina (1995),1.000000
Father of the Bride Part II (1995),0.287343
Grumpier Old Men (1995),0.277298
Waiting to Exhale (1995),0.132838
Sudden Death (1995),0.130281
